In [ ]:
import tensorflow as tf
from tensorflow import keras
import qkeras
import numpy as np
from qkeras.utils import _add_supported_quantized_objects
co = {}
_add_supported_quantized_objects(co)
from tensorflow.keras.models import load_model

import hls4ml
import hls4ml.utils
import hls4ml.converters
from hls4ml.model.profiling import numerical, get_ymodel_keras
from hls4ml.model.optimizer.passes.qkeras import OutputRoundingSaturationMode

import tensorflow as tf
from tensorflow_model_optimization.sparsity.keras import strip_pruning

import json

import os

# Append Vitis HLS 2024.1 bin directory to PATH
os.environ['PATH'] += ':/opt/Xilinx/Vitis_HLS/2024.1/bin'

# start on the server jupyter notebook --no-browser --port=8080 --NotebookApp.token='' --NotebookApp.password='' in the conde env
# run ssh -L 8080:localhost:8080 hlssynt-07.cern.ch

In [ ]:
# get data
data = np.load("testData_200keV_1000samples_unnormalized.npz", allow_pickle=True)
input_images = data["input_images"]
labels = data["labels"]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Flatten all pixel values into one array
pixel_values = input_images.flatten()

# Plot histogram
plt.figure(figsize=(8, 4))
plt.hist(pixel_values, bins=range(int(pixel_values.min()), int(pixel_values.max()) + 2), color='skyblue', edgecolor='black')
plt.title("Histogram of Pixel Values")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.semilogy()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# add custom loss
def l2_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true))
co["Custom>l2_loss"] = l2_loss

In [ ]:
model = load_model("inception_GlobalMaxPool_may26.h5", custom_objects=co)
# model = strip_pruning(model)

In [ ]:
config = hls4ml.utils.config_from_keras_model(model, granularity='name', backend='Vitis', default_precision='ap_ufixed<17,7,AP_RND,AP_SAT>')
config['Model']['ReuseFactor'] = 1
config['Model']['Strategy'] = "Latency"
input_precision = 'ap_ufixed<17,7,AP_RND,AP_SAT>'
config['LayerName']['image_input']['Precision']['result'] = input_precision
config['Flows'] = ['vitis:fifo_depth_optimization']
hls4ml.model.optimizer.get_optimizer('vitis:fifo_depth_optimization').configure(profiling_fifo_depth=200_000)
config['LayerName']['image_input']['Precision']['result'] = input_precision

In [ ]:
hmodel = hls4ml.converters.convert_from_keras_model(
    model,
    hls_config=config,
    backend='Vitis',
    output_dir='version1',
    io_type='io_stream',
    part='xcvu47p-fsvh2892-2L-e'
)

In [ ]:
numerical(model=model, hls_model=hmodel)

In [ ]:
hmodel.compile()
pred_hls = hmodel.predict(input_images)

In [ ]:
pred_qkeras = model.predict(input_images)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

for name, pr in zip(["qkeras", "hls4ml"], [pred_qkeras, pred_hls]):
    diff = labels - pr
    rmse = np.sqrt(np.mean(diff**2))
    print(name, rmse)

    # Plot the predictions vs. true labels
    plt.figure(figsize=(8, 5))
    plt.scatter(labels, pr, alpha=0.5, label=f'Prediction ({name})')
    plt.plot([labels.min(), labels.max()], [labels.min(), labels.max()], 'k--', lw=2, label='Ideal')
    plt.xlabel("True Labels")
    plt.ylabel(f"Predicted ({name})")
    plt.title(f"True vs. Predicted for {name}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Plotting the residuals for both models
plt.figure(figsize=(8, 5))
plt.hist(labels - pred_qkeras, bins=50, alpha=0.5, label='Residuals (qkeras)')
plt.hist(labels - pred_hls, bins=50, alpha=0.5, label='Residuals (hls4ml)')
plt.xlabel("Residual (True - Predicted)")
plt.ylabel("Frequency")
plt.title("Residual Distribution")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
hmodel.compile()